In [34]:
import pdfplumber
import pandas as pd
import numpy as np

In [39]:
def fix_pdf_table(df):
    """
    Fix common PDF table extraction problems:
    - Header split into multiple rows
    - Header split across multiple columns
    - First row mistakenly treated as data
    - Missing/partial header fragments
    """

    df = df.copy()

    # Remove whitespace everywhere
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # If columns are numbers, use row 0 as header
    if all(isinstance(c, int) for c in df.columns):
        if df.iloc[0].apply(lambda x: isinstance(x, str)).mean() > 0.6:
            df.columns = df.iloc[0]
            df = df[1:].reset_index(drop=True)
            return df

    # If row 0 is mostly empty => split header
    if df.iloc[0].isna().sum() > len(df.columns) / 2:
        header = (df.iloc[0].fillna('') + " " + df.iloc[1].fillna('')).str.strip()
        df.columns = header
        df = df[2:].reset_index(drop=True)
        return df

    # If row 0 contains fragments
    if df.iloc[0].astype(str).str.len().mean() < 5:
        header = (df.iloc[0].fillna('') + " " + df.iloc[1].fillna('')).str.strip()
        df.columns = header
        df = df[2:].reset_index(drop=True)
        return df

    # Fallback
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    return df



# ------------------------------------------------------------
# 2. PARSE PDF AND CLEAN TABLES
# ------------------------------------------------------------
def extract_pdf_tables(pdf_path):
    all_tables = []

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):

            tables = page.extract_tables()

            if not tables:
                continue

            for tbl in tables:
                raw_df = pd.DataFrame(tbl)
                clean_df = fix_pdf_table(raw_df)
                all_tables.append(clean_df)

    return all_tables



# ------------------------------------------------------------
# 3. MERGE ALL TABLES (OPTIONAL)
# ------------------------------------------------------------
def merge_tables(table_list):
    """
    Only use this if ALL tables have identical columns.
    """
    return pd.concat(table_list, ignore_index=True)


In [ ]:
pdf_file = "northmet-feis-adequacy-exhibit-a.pdf"   # <-- change this

tables = extract_pdf_tables(pdf_file)

print(f"\nExtracted and cleaned {len(tables)} tables.\n")

# Example: show first cleaned table
if tables:
    print(tables[0].head())

merged = merge_tables(tables)

In [36]:
merged.head()

,Name of,,,Substantive /,Old /,Response,
0,Sender,Comment,Issue,Non-Substantive,New,ID,RGU Consideration
1,Kathleen\nWhitson,PLEASE do NOT approve the mining. It will prof...,GEN,NS,X,1,
2,Mark,The environment will eventually be polluted by...,FIN,NS,X,1,
3,Bob\nWoodbury,Have there been other projects of this nature ...,PER,NS,X,1,
4,Bob\nWoodbury,"I could go on in this vein, but my point is th...",PER,NS,X,1,


In [37]:
ret_columns = []
for i in range(len(merged.columns)):
    ret_columns.append(merged.columns[i] + ' ' + merged.iloc[0][i])

merged.columns = ret_columns

merged = merged[1:].reset_index(drop=True)

merged.head()

['Name of Sender', ' Comment', ' Issue', 'Substantive / Non-Substantive', 'Old / New', 'Response ID', ' RGU Consideration']


/var/folders/0x/k0g9brfx65x_m7n7bqdpmlhm0000gn/T/ipykernel_46317/3150135633.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ret_columns.append(merged.columns[i] + ' ' + merged.iloc[0][i])


,Name of Sender,Comment,Issue,Substantive / Non-Substantive,Old / New,Response ID,RGU Consideration
0,Kathleen\nWhitson,PLEASE do NOT approve the mining. It will prof...,GEN,NS,X,1,
1,Mark,The environment will eventually be polluted by...,FIN,NS,X,1,
2,Bob\nWoodbury,Have there been other projects of this nature ...,PER,NS,X,1,
3,Bob\nWoodbury,"I could go on in this vein, but my point is th...",PER,NS,X,1,
4,Bob\nWoodbury,"We need to make a decision on what we know, no...",NEPA,NS,X,1,


In [38]:
merged.to_csv("northmet-feis-adequacy-exhibit-a.csv", index=False)

In [50]:
pdf_path = "014_appendix_a_response_to_comments_on_the_NorthMet_EIS.pdf"

with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[9]    # or any table page
    print(page.extract_text())


Final Environmental Impact Statement (FEIS)
NorthMet Mining Project and Land Exchange
Table A-3 Cooperating Agency Comments and Responses
Comment ID Comment Response Theme(s)
Comments from the USEPA (Submission ID 47834)
2981 Comment # 1. Spill prevention is an important part To guard against possible adverse effects from spilled ore, PolyMet plans WR 151
of the mitigation for this project. Using new or to refurbish the ore cars, tightening or replacing the couplings and linkages
retrofit side dump rail cars (possibly with to minimize gaps along the hinges and joint areas where spillage would
hydraulic air-operation conversions) should be occur. The quantity of ore that could potentially spill through the door and
considered as part of the mitigation package for hinge gaps of a refurbished ore car is estimated to be 0.20 ton per year. This
the proposed action. Proactive mitigation through is a 97 percent reduction from the originally calculated value of 6.14 tons
the use of updated rai

In [54]:
import pdfplumber
import pandas as pd
import re
from collections import defaultdict

PDF_PATH = "yourfile.pdf"
START_PAGE = 10
END_PAGE = 304

# Strings to remove from every page
REMOVE_HEADERS = [
    "Final Environmental Impact Statement (FEIS)",
    "NorthMet Mining Project and Land Exchange",
]

# Strings to remove only on first page
FIRST_PAGE_REMOVE = [
    "Table A-3 Cooperating Agency Comments and Responses",
    "Comments from the USEPA (Submission ID 47834)",
]

# Regex to detect the true start of a new row
COMMENT_ID_REGEX = r"^\d{4}$"

# Columns based on observed layout
COLUMN_BOUNDS = {
    "Comment ID": (0, 120),
    "Comment": (120, 900),
    "Response": (900, 1700),
    "Theme(s)": (1700, 2300),
}


def clean_lines(lines, first_page=False):
    cleaned = []
    for line in lines:
        if line in REMOVE_HEADERS:
            continue
        if first_page and line in FIRST_PAGE_REMOVE:
            continue
        cleaned.append(line)
    return cleaned


def extract_words_by_column(words):
    """
    Assigns each PDF word to a column based on x0 (left position).
    """
    col_data = {col: [] for col in COLUMN_BOUNDS}
    for w in words:
        x0 = w["x0"]
        text = w["text"]
        for col, (xmin, xmax) in COLUMN_BOUNDS.items():
            if xmin <= x0 < xmax:
                col_data[col].append((w["top"], text))  # Keep y-position to preserve order
                break
    return col_data


def parse_page(page, first_page=False):
    """
    Extracts structured rows from a single page.
    """

    # Extract text raw for header removal
    raw_lines = page.extract_text().split("\n")
    lines = clean_lines(raw_lines, first_page=first_page)

    # Now extract word-level positions
    words = page.extract_words(use_text_flow=True)

    # Group words into columns using x-positions
    col_words = extract_words_by_column(words)

    # Convert each column's word list into sorted text blocks
    col_text = {}
    for col, items in col_words.items():
        # Sort by y-position (top)
        items_sorted = sorted(items, key=lambda x: x[0])
        # Extract just words
        text = " ".join([t for _, t in items_sorted])
        col_text[col] = text

    return col_text


def reconstruct_rows(col_text):
    """
    Reconstructs actual table rows by splitting on Comment ID occurrences.
    """
    rows = []
    cell_ids = re.findall(COMMENT_ID_REGEX, col_text["Comment ID"])

    if not cell_ids:
        return rows

    # Split Comment ID column into rows
    cid_list = re.split(r"\s+", col_text["Comment ID"])
    comment_list = re.split(r"(?=Comment #)", col_text["Comment"])
    response_list = re.split(r"(?=FEIS|Mine|Water|During|This|\w+)", col_text["Response"])
    theme_list = col_text["Theme(s)"].split()

    # Align lengths safely
    max_len = max(len(cid_list), len(comment_list), len(response_list), len(theme_list))
    cid_list += [""] * (max_len - len(cid_list))
    comment_list += [""] * (max_len - len(comment_list))
    response_list += [""] * (max_len - len(response_list))
    theme_list += [""] * (max_len - len(theme_list))

    for i in range(max_len):
        rows.append([
            cid_list[i].strip(),
            comment_list[i].strip(),
            response_list[i].strip(),
            theme_list[i].strip()
        ])

    return rows


# -------------------------------
# MAIN EXTRACTION LOOP
# -------------------------------

all_rows = []
first = True

with pdfplumber.open(PDF_PATH) as pdf:
    for i in range(START_PAGE - 1, END_PAGE):
        page = pdf.pages[i]

        col_text = parse_page(page, first_page=first)
        first = False

        rows = reconstruct_rows(col_text)
        all_rows.extend(rows)

# -------------------------------
# BUILD FINAL DATAFRAME
# -------------------------------

df = pd.DataFrame(all_rows, columns=["Comment ID", "Comment", "Response", "Theme(s)"])

# Remove any accidental blank rows
df = df[df["Comment ID"].str.strip() != ""]

# df.to_csv("NorthMet_TableA3_cleaned.csv", index=False)

print("Extraction complete.")

Page 9: Extracted 29 rows, 1 columns.
Page 10: Extracted 36 rows, 1 columns.
Page 11: Extracted 39 rows, 1 columns.
Page 12: Extracted 40 rows, 1 columns.
Page 13: Extracted 39 rows, 1 columns.
Page 14: Extracted 40 rows, 1 columns.
Page 15: Extracted 38 rows, 1 columns.
Page 16: Extracted 39 rows, 1 columns.
Page 17: Extracted 39 rows, 1 columns.
Page 18: Extracted 41 rows, 1 columns.
Page 19: Extracted 39 rows, 1 columns.
Page 20: Extracted 38 rows, 1 columns.
Page 21: Extracted 65 rows, 1 columns.
Page 22: Extracted 38 rows, 1 columns.
Page 23: Extracted 40 rows, 1 columns.
Page 24: Extracted 39 rows, 1 columns.
Page 25: Extracted 39 rows, 1 columns.
Page 26: Extracted 39 rows, 1 columns.
Page 27: Extracted 39 rows, 1 columns.
Page 28: Extracted 39 rows, 1 columns.
Page 29: Extracted 39 rows, 1 columns.
Page 30: Extracted 39 rows, 1 columns.
Page 31: Extracted 37 rows, 1 columns.
Page 32: Extracted 38 rows, 1 columns.
Page 33: Extracted 38 rows, 1 columns.
Page 34: Extracted 39 rows

In [55]:
merged.head()

,0 The comment evaluation process used a thematic response approach. Subject matter experts from
0,the Co-lead Agencies and their consultants rev...
1,area according to the common topic they addres...
2,theme. Each of the 23 issue areas includes mul...
3,topics addressed by comments. Each comment was...
4,"where a comment addressed more than one theme,..."
